# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 06: Proposed solution** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Lab06-Solution") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 12:50:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Loading Netflix dataset

In [3]:
from team_name.spark_utils import SparkUtils

columns_info = [("show_id", "string"),("type", "string"),("title", "string"),("director", "string"),("country", "string"),("date_added", "string"),("release_year", "integer"),("rating", "string"), ("duration", "string"),("listed_in", "string")]

netflix_schema = SparkUtils.generate_schema(columns_info)

netflix_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

netflix_df.printSchema()

netflix_df.show(5, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead           

In [4]:
from pyspark.sql.functions import count, when, col
df_nulls = netflix_df.select([count(when(col(c).isNull(), c)).alias(c) for c in netflix_df.columns])

df_nulls.collect()

[Row(show_id=0, type=0, title=0, director=1, country=1, date_added=1, release_year=2, rating=1, duration=2, listed_in=2)]

#### Clean data

In [5]:
clean_dict = {
    "director": "Uknown director",
    "country": "Uknown",
    "date_added": "Not provided",
    "release_year": 0,
    "rating": "Uknown",
    "duration": "Uknown",
    "listed_in": "Not provided"
}
netflix_df = SparkUtils.clean_df(netflix_df, clean_dict)

df_nulls = netflix_df.select([count(when(col(c).isNull(), c)).alias(c) for c in netflix_df.columns])

df_nulls.collect()


[Row(show_id=0, type=0, title=0, director=0, country=0, date_added=0, release_year=0, rating=0, duration=0, listed_in=0)]

In [6]:
write_opts_dict = {
    "mode": "overwrite",
    "parition_cols": ["release_year"],
    "format": "csv",
    "path": "/home/jovyan/notebooks/data/netflix_output"

}

SparkUtils.write_df(netflix_df, write_opts_dict)

Done


In [7]:
# Stop the SparkContext
sc.stop()